<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'''
!pip install pandas==2.2.2 -qq
!pip install pyxirr -qq
!pip install ta -qq
!pip install yfinance==0.2.59 -qq
'''

'\n!pip install pandas==2.2.2 -qq\n!pip install pyxirr -qq\n!pip install ta -qq\n!pip install yfinance==0.2.59 -qq\n'

In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  #stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-02-05 15:30:38
-------------------
qualified stocks: 86
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  3.49 K
Current:  1.39 C
-------------------
Today PnL: -55.52 K (-0.4%)
Current PnL: -30.57 L (-19.5%)
CY Booked + Current PnL: -15.83 L (-10.1%)
-------------------
Total profit:  1.21 L
Total loss:  -31.78 L
-------------------
Total Booked + Current PnL: 11.15 L (8.59%)
Total Booked PnL: 41.72 L (32.14%)
Curr Year Booked PnL: 14.74 L (10.61%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 95.96 L (69.07%)
Deployed:  1.30 C
Current:  1.39 C
CAGR/XIRR %: 3.54%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COALINDIA,-0.53,6.38,12.07,19.23,19319.0,9602.0,160062.0,484.83,24.27,55.0,L-LC,12.37,182.0,0.50,1.15,29.01,XY25,ATH,MINING
50,MASFIN,0.75,-3.97,26.59,21.57,25020.0,-3885.0,94095.0,397.04,-17.34,50.0,H-SC,3.49,168.0,-0.16,0.68,37.31,XR,ATH,FINANCE
77,TTKPRESTIG,1.23,-27.01,37.13,0.09,27311.0,-27221.0,73556.0,770.00,70.76,37.0,M-SC,7.87,253.0,-1.00,0.53,1.24,OX40N,NTT,DURABLES
57,RECLTD,0.09,3.40,16.77,20.74,35229.0,6902.0,210072.0,446.00,35.83,60.0,M-MC,3.98,189.0,0.20,1.51,14.37,XY25,NTT,FINANCE
51,MEDANTA,-1.06,-7.24,33.77,24.08,39766.0,-9195.0,117755.0,1486.00,-14.79,48.0,X-SC,11.10,91.0,-0.23,0.85,7.85,XY24,NTT,HEALTHCARE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,VALIANTORG,11.23,-43.43,207.86,74.15,192370.0,-71057.0,92548.0,838.00,-266.56,64.0,H-SC,26.23,145.0,-0.37,0.67,18.63,XR,NTT,CHEMICALS
14,BSOFT,5.25,-15.98,79.46,50.79,93906.0,-22473.0,118180.0,831.70,7.85,65.0,H-SC,6.60,171.0,-0.24,0.85,37.60,XR,ATH,IT
83,WHIRLPOOL,4.36,-33.48,173.49,81.92,132477.0,-38438.0,76360.0,2270.00,-61.01,49.0,M-SC,32.93,218.0,-0.29,0.55,7.26,XR,NTT,DURABLES
52,PAGEIND,2.92,-10.32,45.65,30.62,97043.0,-24474.0,212580.0,51605.08,-36.23,60.0,X-MC,19.99,57.0,-0.25,1.53,8.95,X40,ATH,APPARELS
24,DMART,2.76,-5.53,37.19,29.60,87694.0,-13800.0,235800.0,5391.45,-19.43,63.0,X-LC,9.63,10.0,-0.16,1.70,15.45,X40N,ATH,FMCG


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,-3.53,-48.78,164.11,35.27,95441.0,-55392.0,58157.0,80.00,-59.76,38.0,L-SC,12.89,268.0,-0.58,0.42,42.34,XR,NTT,HOTELS
76,TRIDENT,-3.52,-24.39,69.97,28.51,48806.0,-22502.0,69753.0,48.00,-10.63,60.0,M-SC,9.94,226.0,-0.46,0.50,19.36,XR,NTT,TEXTILES
64,SHALBY,-2.92,-34.43,90.78,25.10,107828.0,-62363.0,118780.0,327.00,774.00,41.0,M-SC,15.02,248.0,-0.58,0.86,4.04,XY24,NTT,HEALTHCARE
23,DIXON,-2.77,-21.28,66.74,31.26,121243.0,-49104.0,181664.0,18931.72,-63.26,50.0,X-MC,27.72,56.0,-0.41,1.31,11.49,X40N,ATH,IT
17,CERA,-2.49,-33.91,87.44,23.89,101660.0,-59640.0,116263.0,9475.00,-36.28,46.0,H-SC,15.94,154.0,-0.59,0.84,3.73,OX40N,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,0.33,-5.34,119.63,107.91,167200.0,-7880.0,139764.0,1641.55,-22.98,51.0,M-SC,11.50,220.0,-0.05,1.01,5.15,OX40N,ATH,CABLES
66,SIS,1.91,-19.16,50.00,21.26,44880.0,-21272.0,89760.0,528.00,2124.35,67.0,H-SC,4.65,163.0,-0.47,0.65,21.27,OX40N,NTT,MISC
72,TATAELXSI,-2.18,-22.32,69.96,32.03,71646.0,-29425.0,102410.0,9161.00,-15.87,48.0,H-SC,6.39,157.0,-0.41,0.74,14.71,OX40N,NTT,IT
77,TTKPRESTIG,1.23,-27.01,37.13,0.09,27311.0,-27221.0,73556.0,770.00,70.76,37.0,M-SC,7.87,253.0,-1.00,0.53,1.24,OX40N,NTT,DURABLES
48,KANSAINER,-1.83,-27.24,55.96,13.47,109794.0,-73467.0,196200.0,340.00,-70.06,36.0,H-SC,5.59,159.0,-0.67,1.41,0.00,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,0.09,3.40,16.77,20.74,35229.0,6902.0,210072.0,446.00,35.83,60.0,M-MC,3.98,189.0,0.20,1.51,14.37,XY25,NTT,FINANCE
18,COALINDIA,-0.53,6.38,12.07,19.23,19319.0,9602.0,160062.0,484.83,24.27,55.0,L-LC,12.37,182.0,0.50,1.15,29.01,XY25,ATH,MINING
1,ABB,-0.10,5.48,38.01,45.57,104889.0,14333.0,275952.0,7934.00,-32.18,71.0,H-MC,5.21,121.0,0.14,1.99,22.54,AR,NTT,ELECTRICAL


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.10,5.48,38.01,45.57,104889.0,14333.0,275952.0,7934.00,-32.18,71.0,H-MC,5.21,121.0,0.14,1.99,22.54,AR,NTT,ELECTRICAL
39,INDIAMART,0.26,-1.98,110.90,106.72,134070.0,-2443.0,120893.0,4810.62,-51.39,58.0,H-SC,8.48,138.0,-0.02,0.87,20.93,AR,ATH,MISC
26,FINCABLES,0.33,-5.34,119.63,107.91,167200.0,-7880.0,139764.0,1641.55,-22.98,51.0,M-SC,11.50,220.0,-0.05,1.01,5.15,OX40N,ATH,CABLES
84,WIPRO,-0.07,-4.36,80.12,72.27,129842.0,-7388.0,162060.0,420.00,-17.88,38.0,M-LC,2.75,101.0,-0.06,1.17,4.17,XR,NTT,IT
85,ZYDUSLIFE,-0.03,-4.57,42.12,35.63,84242.0,-9569.0,200005.0,1286.17,-18.07,54.0,H-MC,6.07,119.0,-0.11,1.44,11.34,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-1.10,-45.11,146.33,35.20,103346.0,-58045.0,70625.0,2096.00,-7.44,25.0,X-SC,20.91,93.0,-0.56,0.51,0.37,X40,NTT,FOOTWEAR
44,ITC,-1.19,-24.00,45.76,10.78,83580.0,-57669.0,182649.0,452.00,-54.64,27.0,X-LC,18.82,1.0,-0.69,1.32,0.60,X40,NTT,FMCG
81,VBL,-1.79,-11.74,53.71,35.67,149730.0,-37068.0,278774.0,671.64,-22.18,27.0,X-LC,0.29,5.0,-0.25,2.01,0.49,X40N,ATH,FMCG
6,ASIANPAINT,-0.78,-14.94,42.19,20.95,90349.0,-37620.0,214148.0,3460.25,-15.81,29.0,X-LC,9.53,36.0,-0.42,1.54,15.37,X40,ATH,PAINTS
42,INFY,-1.03,3.47,29.74,34.24,96286.0,10872.0,323760.0,1972.00,-19.60,32.0,X-LC,6.50,6.0,0.11,2.33,12.04,X40,NTT,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,VBL,-1.79,-11.74,53.71,35.67,149730.0,-37068.0,278774.0,671.64,-22.18,27.0,X-LC,0.29,5.0,-0.25,2.01,0.49,X40N,ATH,FMCG
73,TCS,-0.25,-17.74,44.12,18.55,137274.0,-67100.0,311137.0,4311.58,-29.13,37.0,X-LC,0.48,3.0,-0.49,2.24,5.85,X40,ATH,IT
5,ANGELONE,-1.64,8.22,14.77,24.20,48791.0,25088.0,330338.0,3033.00,54.39,56.0,X-SC,1.43,99.0,0.51,2.38,38.35,X40N,NTT,FINANCE
34,HINDUNILVR,-0.63,-8.35,24.02,13.67,56591.0,-21463.0,235600.0,2922.00,-15.28,47.0,X-LC,2.36,9.0,-0.38,1.70,11.22,XY25,NTT,FMCG
21,DABUR,0.78,-1.25,45.52,43.70,111512.0,-3094.0,244974.0,735.00,-9.42,46.0,X-MC,2.40,73.0,-0.03,1.76,13.62,XY24,BTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BATAINDIA,-1.10,-45.11,146.33,35.20,103346.0,-58045.0,70625.0,2096.00,-7.44,25.0,X-SC,20.91,93.0,-0.56,0.51,0.37,X40,NTT,FOOTWEAR
81,VBL,-1.79,-11.74,53.71,35.67,149730.0,-37068.0,278774.0,671.64,-22.18,27.0,X-LC,0.29,5.0,-0.25,2.01,0.49,X40N,ATH,FMCG
44,ITC,-1.19,-24.00,45.76,10.78,83580.0,-57669.0,182649.0,452.00,-54.64,27.0,X-LC,18.82,1.0,-0.69,1.32,0.60,X40,NTT,FMCG
2,ABBOTINDIA,0.04,-9.05,29.82,18.07,48505.0,-16195.0,162660.0,35195.00,-22.07,42.0,X-MC,9.32,64.0,-0.33,1.17,1.78,X40,ATH,PHARMA
3,ACC,-1.13,-29.69,133.47,64.15,223295.0,-70651.0,167300.0,3906.00,-58.93,43.0,X-SC,7.78,84.0,-0.32,1.20,2.26,XY24,BTT,CEMENT


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,0.92,-26.22,93.61,42.84,44896.0,-17045.0,47961.0,424.00,-51.52,62.0,X-SC,21.15,83.0,-0.38,0.35,12.74,XY24,NTT,MISC
58,RELAXO,-1.06,-51.82,216.13,52.31,151153.0,-75224.0,69936.0,1176.00,-48.70,42.0,X-SC,12.34,92.0,-0.50,0.50,4.00,X40N,NTT,FOOTWEAR
12,BATAINDIA,-1.10,-45.11,146.33,35.20,103346.0,-58045.0,70625.0,2096.00,-7.44,25.0,X-SC,20.91,93.0,-0.56,0.51,0.37,X40,NTT,FOOTWEAR
51,MEDANTA,-1.06,-7.24,33.77,24.08,39766.0,-9195.0,117755.0,1486.00,-14.79,48.0,X-SC,11.10,91.0,-0.23,0.85,7.85,XY24,NTT,HEALTHCARE
35,HONAUT,-1.64,-20.23,79.67,43.32,103507.0,-32952.0,129920.0,58357.33,-31.51,44.0,X-SC,9.19,90.0,-0.32,0.94,5.49,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,-1.19,-24.00,45.76,10.78,83580.0,-57669.0,182649.0,452.00,-54.64,27.0,X-LC,18.82,1.0,-0.69,1.32,0.60,X40,NTT,FMCG
75,TMPV,0.04,-23.02,59.74,22.98,103890.0,-51995.0,173903.0,600.00,-20.63,66.0,X-LC,4.30,2.0,-0.50,1.25,11.17,XY24,NTT,AUTO
73,TCS,-0.25,-17.74,44.12,18.55,137274.0,-67100.0,311137.0,4311.58,-29.13,37.0,X-LC,0.48,3.0,-0.49,2.24,5.85,X40,ATH,IT
81,VBL,-1.79,-11.74,53.71,35.67,149730.0,-37068.0,278774.0,671.64,-22.18,27.0,X-LC,0.29,5.0,-0.25,2.01,0.49,X40N,ATH,FMCG
42,INFY,-1.03,3.47,29.74,34.24,96286.0,10872.0,323760.0,1972.00,-19.60,32.0,X-LC,6.50,6.0,0.11,2.33,12.04,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,-0.85,-32.56,97.04,32.89,50529.0,-25136.0,52070.0,1800.00,-743.10,56.0,L-MC,11.22,259.0,-0.50,0.37,43.40,XR,NTT,BANKS
67,SONACOMS,-1.66,-9.35,53.44,39.09,49004.0,-9459.0,91700.0,804.02,-29.16,68.0,M-SC,1.82,227.0,-0.19,0.66,29.73,AR,ATH,AUTO
14,BSOFT,5.25,-15.98,79.46,50.79,93906.0,-22473.0,118180.0,831.70,7.85,65.0,H-SC,6.60,171.0,-0.24,0.85,37.60,XR,ATH,IT
18,COALINDIA,-0.53,6.38,12.07,19.23,19319.0,9602.0,160062.0,484.83,24.27,55.0,L-LC,12.37,182.0,0.50,1.15,29.01,XY25,ATH,MINING
65,SIEMENS,0.22,-11.40,41.66,25.51,68691.0,-21210.0,164885.0,4671.50,1.14,68.0,H-LC,3.60,50.0,-0.31,1.19,21.51,AR,ATH,ELECTRICAL


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,FTT,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
1,ABB,-0.10,5.48,38.01,45.57,104889.0,14333.0,275952.0,7934.00,-32.18,71.0,H-MC,5.21,121.0,0.14,1.99,22.54,AR,NTT,ELECTRICAL
5,ANGELONE,-1.64,8.22,14.77,24.20,48791.0,25088.0,330338.0,3033.00,54.39,56.0,X-SC,1.43,99.0,0.51,2.38,38.35,X40N,NTT,FINANCE
67,SONACOMS,-1.66,-9.35,53.44,39.09,49004.0,-9459.0,91700.0,804.02,-29.16,68.0,M-SC,1.82,227.0,-0.19,0.66,29.73,AR,ATH,AUTO
62,SATIN,0.46,-11.79,72.12,51.83,175771.0,-32564.0,243720.0,274.00,-21.16,64.0,H-SC,3.25,148.0,-0.19,1.75,18.73,XY24,NTT,FINANCE
14,BSOFT,5.25,-15.98,79.46,50.79,93906.0,-22473.0,118180.0,831.70,7.85,65.0,H-SC,6.60,171.0,-0.24,0.85,37.60,XR,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.29
1,25,44.63
2,50,76.63


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.41
MC    30.87
LC    23.76
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.56
X40      23.42
X40N     15.06
AR        9.08
XR        8.84
XY25      8.76
OX40N     7.42
SR        0.90
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.10
H-SC    23.00
X-LC    20.25
M-SC    11.32
X-SC    10.33
H-MC     4.89
M-MC     1.51
H-LC     1.19
M-LC     1.17
L-LC     1.15
L-SC     0.76
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.87,-14.77,53.33
FINANCE,13.49,-11.48,55.35
IT,12.42,-24.01,85.97
MISC,7.23,-29.70,82.76
ELECTRICAL,6.16,-9.84,49.92
PAINTS,5.23,-26.92,45.77
INSURANCE,4.79,-1.97,36.86
PHARMA,4.01,-5.58,38.81
AUTO,2.97,-26.98,70.89


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3282870.0,21
AR,1311611.0,10
XR,1263437.0,12
X40,1245696.0,15
X40N,1031765.0,10
OX40N,762281.0,10
XY25,402851.0,6
SR,295305.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3600941.0,24
X-MC,1679168.0,16
M-SC,1487188.0,15
X-LC,1080731.0,12
X-SC,858783.0,9
H-MC,400383.0,3
L-SC,185012.0,2
M-LC,129842.0,1
H-LC,68691.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1240790.0      6
           AR         932144.0      5
M-SC       XY24       837910.0      6
H-SC       XR         806342.0      7
X-MC       X40        554330.0      7
X-LC       X40        516354.0      6
X-MC       XY24       480408.0      3
           X40N       418527.0      4
X-SC       X40N       375814.0      4
M-SC       OX40N      346350.0      5
H-SC       OX40N      326360.0      4
X-SC       XY24       307957.0      3
H-SC       SR         295305.0      2
X-LC       X40N       237424.0      2
X-MC       XY25       225903.0      2
H-MC       XY24       211252.0      1
X-LC       XY24       204553.0      2
H-MC       AR         189131.0      2
M-SC       XR         181283.0      2
X-SC       X40        175012.0      2
M-LC       XR         129842.0      1
X-LC       XY25       122400.0      2
M-SC       AR         121645.0      2
L-SC       XR          95441.0      1
           OX40N       89571.0      1
H-LC       AR          68691.0      1
L-MC       XR          50529.0      1
M-MC       XY25        35229.0      1
L-LC       XY25        19319.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
